In [175]:
import pandas as pd
import numpy as np
import os,sys
import pickle
from collections import Counter
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras import Model
import sklearn
from sklearn.utils import class_weight
from sklearn.decomposition import *
import string
tf.__version__

'2.1.0'

In [3]:
df = pickle.load(open('../../spark_data/data/Leo_Spark_Home/tmp/part_df.pkl','rb'))
df

,city,domain,frame.len,frame.number,frame.protocols,frame.time,frame.time_epoch,ip.dst,ip.proto,ip.src,...,tcp.flags.ack,tcp.flags.fin,tcp.flags.push,tcp.flags.reset,tcp.flags.syn,tcp.flags.urg,tcp.srcport,tcp.window_size_value,traits_network,country
0,None,maxko-hosting.com,[45],[138533],[eth:ethertype:ip:udp:data],"[Jan 9, 2020 18:20:43.203422000 CST]",[1578565243.203422000],[210.242.145.4],[17],[45.95.168.116],...,None,None,None,None,None,None,None,None,45.95.168.0/22,Croatia
1,None,maxko-hosting.com,[45],[138534],[eth:ethertype:ip:udp:data],"[Jan 9, 2020 18:20:43.203554000 CST]",[1578565243.203554000],[210.242.145.36],[17],[45.95.168.116],...,None,None,None,None,None,None,None,None,45.95.168.0/22,Croatia
2,None,maxko-hosting.com,[45],[138535],[eth:ethertype:ip:udp:data],"[Jan 9, 2020 18:20:43.203610000 CST]",[1578565243.203610000],[210.242.145.37],[17],[45.95.168.116],...,None,None,None,None,None,None,None,None,45.95.168.0/22,Croatia
3,None,maxko-hosting.com,[45],[138536],[eth:ethertype:ip:udp:data],"[Jan 9, 2020 18:20:43.203639000 CST]",[1578565243.203639000],[210.242.145.66],[17],[45.95.168.116],...,None,None,None,None,None,None,None,None,45.95.168.0/22,Croatia
4,None,maxko-hosting.com,[45],[138537],[eth:ethertype:ip:udp:data],"[Jan 9, 2020 18:20:43.203658000 CST]",[1578565243.203658000],[210.242.145.58],[17],[45.95.168.116],...,None,None,None,None,None,None,None,None,45.95.168.0/22,Croatia
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214748375605,Tbilisi,magticom.ge,[66],[634758],[eth:ethertype:ip:tcp],"[Jan 9, 2020 10:03:13.909999000 CST]",[1578535393.909999000],[93.177.147.200],[6],[210.242.148.29],...,[1],[0],[0],[0],[1],[0],[3389],[29200],93.177.144.0/22,Georgia
214748375606,Tbilisi,magticom.ge,[66],[634759],[eth:ethertype:ip:tcp],"[Jan 9, 2020 10:03:13.910020000 CST]",[1578535393.910020000],[93.177.147.200],[6],[210.242.145.13],...,[1],[0],[0],[0],[1],[0],[3389],[29200],93.177.144.0/22,Georgia
214748375607,Tbilisi,magticom.ge,[66],[634761],[eth:ethertype:ip:tcp],"[Jan 9, 2020 10:03:13.910031000 CST]",[1578535393.910031000],[93.177.147.200],[6],[210.242.148.62],...,[1],[0],[0],[0],[1],[0],[3389],[29200],93.177.144.0/22,Georgia
214748375608,Tbilisi,magticom.ge,[66],[634762],[eth:ethertype:ip:tcp],"[Jan 9, 2020 10:03:13.910041000 CST]",[1578535393.910041000],[93.177.147.200],[6],[210.242.148.31],...,[1],[0],[0],[0],[1],[0],[3389],[29200],93.177.144.0/22,Georgia


In [4]:
df.columns

Index(['city', 'domain', 'frame.len', 'frame.number', 'frame.protocols',
       'frame.time', 'frame.time_epoch', 'ip.dst', 'ip.proto', 'ip.src',
       'ip.ttl', 'isp', 'latitude', 'longitude', 'network', 'subdivisions',
       'tcp.dstport', 'tcp.flags.ack', 'tcp.flags.fin', 'tcp.flags.push',
       'tcp.flags.reset', 'tcp.flags.syn', 'tcp.flags.urg', 'tcp.srcport',
       'tcp.window_size_value', 'traits_network', 'country'],
      dtype='object')

### frame.protocols

In [41]:
df2 = df[['frame.protocols']]
df2

,frame.protocols
0,[eth:ethertype:ip:udp:data]
1,[eth:ethertype:ip:udp:data]
2,[eth:ethertype:ip:udp:data]
3,[eth:ethertype:ip:udp:data]
4,[eth:ethertype:ip:udp:data]
...,...
214748375605,[eth:ethertype:ip:tcp]
214748375606,[eth:ethertype:ip:tcp]
214748375607,[eth:ethertype:ip:tcp]
214748375608,[eth:ethertype:ip:tcp]


In [15]:
def protocol_convert(s):
    '''
    s:
    '''
    ss = s[0].split(":")[3:]
    de_re = []
    last = None
    for sss in ss:
        if sss != last:
            de_re.append(sss)
            last = sss
    return " ".join(de_re)

In [42]:
df2['protocols'] = df2['frame.protocols'].map(protocol_convert)
all_pro = df2['protocols'].tolist()
all_pro = sorted(list(set(all_pro)))
print('subtype categories#:',len(all_pro))
all_terms = []
length = []
for pro in all_pro:
    all_terms.extend(pro.split())
    length.append(len(pro.split()))
all_terms = sorted(list(set(all_terms)))
print("terms categories#:",len(all_terms))
all_dict = {}
for i,v in enumerate(all_terms):
    all_dict[v]=i+1
print("Protocol Max length =",max(length))
all_dict

subtype categories#: 27
terms categories#: 20
Protocol Max length = 6


/home/antslab/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


{'data': 1,
 'data-text-lines': 2,
 'ftp': 3,
 'gss-api': 4,
 'http': 5,
 'icmp': 6,
 'ip': 7,
 'irc': 8,
 'memcache': 9,
 'nbss': 10,
 'ntlmssp': 11,
 'ntp': 12,
 'smb': 13,
 'spnego': 14,
 'ssh': 15,
 'tcp': 16,
 'tds': 17,
 'udp': 18,
 'urlencoded-form': 19,
 'xml': 20}

In [46]:
def protocols_encode(s):
    '''
    s: df.protocol (Series)
    '''
    s_ = s.split()
    if len(s_)>1:
        temp = []
        for ss in s_:
            temp.append(all_dict[ss])
        return temp[1:]
    else:
        return [all_dict[s_[0]]]

In [47]:
df2['protocols_encode'] = df2.protocols.map(protocols_encode)
all_pro = df2.protocols_encode.tolist()
df2

/home/antslab/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,frame.protocols,protocols,protocols_encode
0,[eth:ethertype:ip:udp:data],udp data,[1]
1,[eth:ethertype:ip:udp:data],udp data,[1]
2,[eth:ethertype:ip:udp:data],udp data,[1]
3,[eth:ethertype:ip:udp:data],udp data,[1]
4,[eth:ethertype:ip:udp:data],udp data,[1]
...,...,...,...
214748375605,[eth:ethertype:ip:tcp],tcp,[16]
214748375606,[eth:ethertype:ip:tcp],tcp,[16]
214748375607,[eth:ethertype:ip:tcp],tcp,[16]
214748375608,[eth:ethertype:ip:tcp],tcp,[16]


In [48]:
all_pro_np = tf.keras.preprocessing.sequence.pad_sequences(all_pro,dtype='int32', padding='post')
assert all_pro_np.shape[1] == max(length)-1
assert np.max(all_pro_np) == max(all_dict.values())
all_pro_np.shape #trainX

(100000, 5)

In [50]:
def convert_pro_y(s):
    '''
    s: protocols (String series)
    0=>tcp
    1=>udp
    2=>icmp
    (sparse categorical cross entropy)
    '''
    if 'tcp' in s:
        return 0
    elif 'udp' in s:
        return 1
    elif 'icmp' in s:
        return 2
    else:
        return 'ERR'

In [72]:
df2['protocols_ans'] = df2.protocols.map(convert_pro_y)
all_pro_ans = df2['protocols_ans'].tolist()
class_weight_ = dict(Counter(all_pro_ans))
print(class_weight_)
all_pro_ans = np.array(all_pro_ans)
assert all_pro_ans.shape[0]==all_pro_np.shape[0]

{1: 2870, 0: 97100, 2: 30}


/home/antslab/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [74]:
class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(all_pro_ans),
                                                 all_pro_ans)
class_weights

array([3.43288706e-01, 1.16144019e+01, 1.11111111e+03])

In [67]:
embed_dim = 4
class protol_model(Model):
    def __init__(self):
        super(protol_model,self).__init__()
        self.emb = Embedding(np.max(all_pro_np)+1,embed_dim,input_length=all_pro_np.shape[1],mask_zero=True)
        self.rnn = Bidirectional(LSTM(int(embed_dim/2),recurrent_dropout=0.2,dropout=0.2))
        self.out = Dense(max(all_pro_ans)+1,activation='softmax')
    def call(self,x):
        x = self.emb(x)
        x1 = self.rnn(x)
        x = self.out(x1)
        return x
model = protol_model()
model(all_pro_np[:2])

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[0.3322379 , 0.336236  , 0.33152616],
       [0.3322379 , 0.336236  , 0.33152616]], dtype=float32)>

In [85]:
loss_fn= tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
# class_weights = tf.constant(list(class_weights))
# class_weights = tf.compat.v2.constant(list(class_weights))
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])
model.summary()

Model: "protol_model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      multiple                  84        
_________________________________________________________________
lstm_5 (LSTM)                multiple                  144       
_________________________________________________________________
dense_5 (Dense)              multiple                  15        
Total params: 243
Trainable params: 243
Non-trainable params: 0
_________________________________________________________________


In [ ]:
H = model.fit(all_pro_np, all_pro_ans, epochs=5)

In [ ]:
# sum all emb layer & lstm putput

### isp

In [84]:
df3 = df.loc[:,'isp']
df3 = pd.DataFrame(df3)
df3

,isp
0,Giganet Internet Szolgaltato Kft
1,Giganet Internet Szolgaltato Kft
2,Giganet Internet Szolgaltato Kft
3,Giganet Internet Szolgaltato Kft
4,Giganet Internet Szolgaltato Kft
...,...
214748375605,Magticom Ltd.
214748375606,Magticom Ltd.
214748375607,Magticom Ltd.
214748375608,Magticom Ltd.


In [101]:
def get_tokens(s):
    '''
    s: str with space split (isp)(series)
    return: list
    '''
    ss = s.translate(str.maketrans('', '', string.punctuation))
    return ss.lower()#.split()

In [111]:
df3['isp_tokens_li'] = df3.isp.map(get_tokens)
isp_tokens = df3['isp_tokens_li'].unique()
isp_tokens = sorted(list(set(" ".join(isp_tokens).split()))) #all dict
# isp_tokens = sum(isp_tokens,[])
isp_dict = {}
for i,v in enumerate(isp_tokens):
    isp_dict[v]=i+1
isp_dict #save pkl?

{'a1': 1,
 'academic': 2,
 'aldagi': 3,
 'alfa': 4,
 'and': 5,
 'arsinform': 6,
 'broadband': 7,
 'bt': 8,
 'buzton': 9,
 'caucasus': 10,
 'cgc': 11,
 'co': 12,
 'company': 13,
 'council': 14,
 'croatian': 15,
 'deltanet': 16,
 'department': 17,
 'doo': 18,
 'east': 19,
 'energy': 20,
 'erty': 21,
 'fixed': 22,
 'free': 23,
 'freenet': 24,
 'gals': 25,
 'georgia': 26,
 'giganet': 27,
 'global': 28,
 'high': 29,
 'hrvatska': 30,
 'hrvatski': 31,
 'i': 32,
 'image': 33,
 'imax': 34,
 'informservice': 35,
 'institute': 36,
 'insurance': 37,
 'internet': 38,
 'iplus': 39,
 'iskon': 40,
 'isp': 41,
 'istv': 42,
 'jc': 43,
 'joint': 44,
 'jsc': 45,
 'justice': 46,
 'jv': 47,
 'kft': 48,
 'lcc': 49,
 'lepl': 50,
 'link': 51,
 'littel': 52,
 'llc': 53,
 'ltd': 54,
 'magic': 55,
 'magticom': 56,
 'maxnet': 57,
 'media': 58,
 'mobile': 59,
 'nano': 60,
 'net': 61,
 'network': 62,
 'odjel': 63,
 'odrex': 64,
 'of': 65,
 'online': 66,
 'ooo': 67,
 'optima': 68,
 'pe': 69,
 'physics': 70,
 'railway

In [114]:
def encode_tokens(s):
    '''
    s: isp_tokens (str)
    '''
    tokens_int = []
    for token in s.split():
        tokens_int.append(isp_dict[token])
    return tokens_int

In [116]:
df3['isp_encode'] = df3.isp_tokens_li.map(encode_tokens) #轉換token
isp_encode_np = df3.isp_encode.tolist()
isp_encode_np = tf.keras.preprocessing.sequence.pad_sequences(isp_encode_np,dtype='int32', padding='post') #計算maxlen、padding
assert np.max(isp_encode_np) == max(isp_dict.values())
isp_encode_np.shape #trainX

(100000, 8)

In [118]:
ori_dict = dict(Counter(df3.isp.tolist()))
max_num = max(ori_dict.values())
min_num = min(ori_dict.values())
range_ = max_num-min_num
new_dict = {}
for k,v in ori_dict.items():
    new_dict[k]=(v-min_num)/(range_)
new_dict    

{'Giganet Internet Szolgaltato Kft': 0.09918022926083146,
 'Tele2 SWIPnet': 0.0015295512743396292,
 'Hrvatski Telekom - fixed broadband': 0.010758416828501212,
 'VIPNET': 0.00807740560606321,
 'Iskon Internet': 0.0003437193874920515,
 'A1 Hrvatska': 0.008094591575437812,
 'Croatian Academic and Research Network': 0.002784127038685617,
 'BT NET d.o.o. za trgovinu i usluge': 5.155790812380772e-05,
 'Optima Telekom': 0.000807740560606321,
 'Magic Net D.o.o.': 0.0025263374980665785,
 'Sedmi Odjel d.o.o.': 0.0,
 'NANO TELECOM Ltd': 0.044408544863973054,
 'Uzbektelekom Joint Stock Company': 0.011463041572859918,
 'JC LLC Sarkor-Telecom': 0.003969958925533194,
 'East Telecom': 0.01622355508962483,
 'Turon Telecom': 0.0019935724474538985,
 'LLC texnoprosistem': 0.003591867599291938,
 'OOO Gals Telecom': 0.0005155790812380772,
 'UNITEL LLC': 0.0008592984687301287,
 'Turon Media XK': 0.0009108563768539364,
 'teleport LLC': 0.0021654321411999243,
 'Super iMAX': 0.0008249265299809235,
 'Uzbektelec

In [123]:
df3['isp_ans'] = df3.isp.map(new_dict)#計算各isp詞頻答案
isp_ans_np = df3.isp_ans.to_numpy()
assert isp_ans_np.shape[0]==isp_encode_np.shape[0]
isp_ans_np.shape

(100000,)

In [137]:
embed_dim = 3
class isp_model(Model):
    def __init__(self):
        super(isp_model,self).__init__()
        self.emb = Embedding(np.max(isp_encode_np)+1,embed_dim,mask_zero=True,
                             input_length=isp_encode_np.shape[1])
        self.rnn = GRU(embed_dim,recurrent_dropout=0.2,dropout=0.2)
        self.out = Dense(1)
    def call(self,x):
        x = self.emb(x)
        x1 = self.rnn(x)
        x = self.out(x1)
        return x
model = isp_model()

model(isp_encode_np[:2])

#build model(embedding layer, rnn ,dense)=>regression

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[-0.00819251],
       [-0.00819251]], dtype=float32)>

In [138]:
model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
model.summary()


Model: "isp_model_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_18 (Embedding)     multiple                  309       
_________________________________________________________________
gru_12 (GRU)                 multiple                  72        
_________________________________________________________________
dense_18 (Dense)             multiple                  4         
Total params: 385
Trainable params: 385
Non-trainable params: 0
_________________________________________________________________


In [139]:
H = model.fit(isp_encode_np,isp_ans_np,epochs=10)

Train on 100000 samples
Epoch 1/5
100000/100000 [==============================] - 11s 113us/sample - loss: 0.0227 - mae: 0.0752
Epoch 2/5
100000/100000 [==============================] - 10s 99us/sample - loss: 9.6220e-04 - mae: 0.0189
Epoch 3/5
100000/100000 [==============================] - 10s 98us/sample - loss: 4.8556e-04 - mae: 0.0125
Epoch 4/5
100000/100000 [==============================] - 10s 98us/sample - loss: 3.7071e-04 - mae: 0.0104
Epoch 5/5
100000/100000 [==============================] - 10s 99us/sample - loss: 3.1647e-04 - mae: 0.0097


### Country、City、Subdivisions

In [166]:
df4 = df[['country','city','subdivisions']]
df4 = df4.fillna('<UNK>')
df4

,country,city,subdivisions
0,Croatia,<UNK>,<UNK>
1,Croatia,<UNK>,<UNK>
2,Croatia,<UNK>,<UNK>
3,Croatia,<UNK>,<UNK>
4,Croatia,<UNK>,<UNK>
...,...,...,...
214748375605,Georgia,Tbilisi,K'alak'i T'bilisi
214748375606,Georgia,Tbilisi,K'alak'i T'bilisi
214748375607,Georgia,Tbilisi,K'alak'i T'bilisi
214748375608,Georgia,Tbilisi,K'alak'i T'bilisi


In [171]:
df_country = pd.get_dummies(df4['country'])
df41 = df4.join(df_country)
len1 = len(df_country.columns)
df_city = pd.get_dummies(df41['city'])
df42 = df41.join(df_city)
len2 = len(df_city.columns)
df_sub = pd.get_dummies(df42['subdivisions'])
df43 = df42.join(df_sub, lsuffix='_left', rsuffix='_right')
len3 = len(df_sub.columns)
assert len1+len2+len3+3 == len(df43.columns)
df43

,country,city,subdivisions,Croatia,Georgia,Uzbekistan,<UNK>_left,Andijan_left,Antin,Batumi,...,Primorsko-Goranska Zupanija,Sibensko-Kninska Zupanija,Slavonski Brod-Posavina,Split-Dalmatia,Toshkent Shahri,Varazdinska Zupanija,Vukovar-Sirmium,Xorazm Viloyati,Zadarska Zupanija,Zagreb County
0,Croatia,<UNK>,<UNK>,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Croatia,<UNK>,<UNK>,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Croatia,<UNK>,<UNK>,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Croatia,<UNK>,<UNK>,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Croatia,<UNK>,<UNK>,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214748375605,Georgia,Tbilisi,K'alak'i T'bilisi,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
214748375606,Georgia,Tbilisi,K'alak'i T'bilisi,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
214748375607,Georgia,Tbilisi,K'alak'i T'bilisi,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
214748375608,Georgia,Tbilisi,K'alak'i T'bilisi,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [173]:
df_encode = df43.iloc[:,3:]
df_encode = df_encode.to_numpy()
df_encode.shape

(100000, 64)

In [181]:
pca = PCA(n_components=3,iterated_power='auto', whiten=True,svd_solver="full") #n_components='mle',395
encode_npy = pca.fit_transform(df_encode)
encode_npy.shape #final encoding result (city country subdivision)

(100000, 2)

### domain

In [209]:
df5 = df.loc[:,'domain']
df5 = pd.DataFrame(df5)
df5 = df5.fillna('<NAN>')
df5

,domain
0,maxko-hosting.com
1,maxko-hosting.com
2,maxko-hosting.com
3,maxko-hosting.com
4,maxko-hosting.com
...,...
214748375605,magticom.ge
214748375606,magticom.ge
214748375607,magticom.ge
214748375608,magticom.ge


In [214]:
def convert_domain_li(s):
    '''
    s: domain series(str) split by "."
    '''
    return s.split('.')
def calc_length(s):
    '''
    calculate list's length in Series
    '''
    return len(s)
def prepend_li(s):
    '''
    prepend <NAN> to max_num
    '''
    ss = s[:]
    if len(ss)<max_num:
        iter_num = max_num-len(ss)
        for _ in range(iter_num):
            ss.insert(0,'<NAN>')
    return ss

In [215]:
df5['domain_li'] = df5.domain.map(convert_domain_li)
df5['domain_length'] = df5.domain_li.map(calc_length)
max_num = df5.domain_length.max()
print(max_num)
df5['domain_li_rev'] = df5.domain_li.map(prepend_li)
df5

3


,domain,domain_li,domain_length,domain_li_rev
0,maxko-hosting.com,"[maxko-hosting, com]",2,"[<NAN>, maxko-hosting, com]"
1,maxko-hosting.com,"[maxko-hosting, com]",2,"[<NAN>, maxko-hosting, com]"
2,maxko-hosting.com,"[maxko-hosting, com]",2,"[<NAN>, maxko-hosting, com]"
3,maxko-hosting.com,"[maxko-hosting, com]",2,"[<NAN>, maxko-hosting, com]"
4,maxko-hosting.com,"[maxko-hosting, com]",2,"[<NAN>, maxko-hosting, com]"
...,...,...,...,...
214748375605,magticom.ge,"[magticom, ge]",2,"[<NAN>, magticom, ge]"
214748375606,magticom.ge,"[magticom, ge]",2,"[<NAN>, magticom, ge]"
214748375607,magticom.ge,"[magticom, ge]",2,"[<NAN>, magticom, ge]"
214748375608,magticom.ge,"[magticom, ge]",2,"[<NAN>, magticom, ge]"


In [216]:
col_name = []
for i in range(max_num):
    col_name.append('slot'+str(i+1))
df51 = pd.DataFrame(df5['domain_li_rev'].tolist(),columns=col_name)
df51

,slot1,slot2,slot3
0,<NAN>,maxko-hosting,com
1,<NAN>,maxko-hosting,com
2,<NAN>,maxko-hosting,com
3,<NAN>,maxko-hosting,com
4,<NAN>,maxko-hosting,com
...,...,...,...
99995,<NAN>,magticom,ge
99996,<NAN>,magticom,ge
99997,<NAN>,magticom,ge
99998,<NAN>,magticom,ge


In [219]:
for col in df51.columns:
    temp = dict(Counter(df51[col].tolist()))
    max_num = max(temp.values())
    min_num = min(temp.values())
    range_ = max_num-min_num
    def min_max_norm(s):
        ss = int(temp[s])
        ss = (ss-min_num)/range_
        return ss
    df51[col+'_enc'] = df51[col].map(min_max_norm)
df51    

,slot1,slot2,slot3,slot1_enc,slot2_enc,slot3_enc
0,<NAN>,maxko-hosting,com,1.0,0.111302,0.073973
1,<NAN>,maxko-hosting,com,1.0,0.111302,0.073973
2,<NAN>,maxko-hosting,com,1.0,0.111302,0.073973
3,<NAN>,maxko-hosting,com,1.0,0.111302,0.073973
4,<NAN>,maxko-hosting,com,1.0,0.111302,0.073973
...,...,...,...,...,...,...
99995,<NAN>,magticom,ge,1.0,1.000000,1.000000
99996,<NAN>,magticom,ge,1.0,1.000000,1.000000
99997,<NAN>,magticom,ge,1.0,1.000000,1.000000
99998,<NAN>,magticom,ge,1.0,1.000000,1.000000


In [223]:
col_li = df51.columns
col_li = [x for x in col_li if '_enc' in x]
domain_enc = df51[col_li].to_numpy()
domain_enc.shape #final encoding result (domain)

(100000, 3)

### tcp.dstport、tcp.srcport
* 要改attribute name

In [226]:
df6 = df[['tcp.dstport','tcp.srcport']]
df6 = df6.fillna(-1)
df6

,tcp.dstport,tcp.srcport
0,-1,-1
1,-1,-1
2,-1,-1
3,-1,-1
4,-1,-1
...,...,...
214748375605,[59305],[3389]
214748375606,[59304],[3389]
214748375607,[59332],[3389]
214748375608,[59317],[3389]


In [229]:
def convert_port(s):
    '''
    if -1: return [-1,-1,-1,-1]
    else minMax norm & category one-hot
    '''
    if s == -1:
        return [-1,-1,-1,-1]
    else:
        ss = int(s[0])
        minmax = [(ss-1)/65534]
        if ss <1024:
            minmax.extend([1,0,0])
        elif ss<49152:
            minmax.extend([0,1,0])
        else:
            minmax.extend([0,0,1])
        return minmax

In [231]:
df6['port1'] = df6['tcp.dstport'].map(convert_port)
df6['port2'] = df6['tcp.srcport'].map(convert_port)
df6['ports'] = df6['port1']+df6['port2']
df6

,tcp.dstport,tcp.srcport,port1,port2,ports
0,-1,-1,"[-1, -1, -1, -1]","[-1, -1, -1, -1]","[-1, -1, -1, -1, -1, -1, -1, -1]"
1,-1,-1,"[-1, -1, -1, -1]","[-1, -1, -1, -1]","[-1, -1, -1, -1, -1, -1, -1, -1]"
2,-1,-1,"[-1, -1, -1, -1]","[-1, -1, -1, -1]","[-1, -1, -1, -1, -1, -1, -1, -1]"
3,-1,-1,"[-1, -1, -1, -1]","[-1, -1, -1, -1]","[-1, -1, -1, -1, -1, -1, -1, -1]"
4,-1,-1,"[-1, -1, -1, -1]","[-1, -1, -1, -1]","[-1, -1, -1, -1, -1, -1, -1, -1]"
...,...,...,...,...,...
214748375605,[59305],[3389],"[0.9049348429822688, 0, 0, 1]","[0.051698355052339245, 0, 1, 0]","[0.9049348429822688, 0, 0, 1, 0.05169835505233..."
214748375606,[59304],[3389],"[0.9049195837275308, 0, 0, 1]","[0.051698355052339245, 0, 1, 0]","[0.9049195837275308, 0, 0, 1, 0.05169835505233..."
214748375607,[59332],[3389],"[0.9053468428601947, 0, 0, 1]","[0.051698355052339245, 0, 1, 0]","[0.9053468428601947, 0, 0, 1, 0.05169835505233..."
214748375608,[59317],[3389],"[0.9051179540391248, 0, 0, 1]","[0.051698355052339245, 0, 1, 0]","[0.9051179540391248, 0, 0, 1, 0.05169835505233..."


In [234]:
df61 = pd.DataFrame(df6['ports'].tolist(),columns=None)
ports_enc = df61.to_numpy()
ports_enc.shape  #final encoding result (2 ports)

(100000, 8)

In [ ]:
#flags、proto
#windows size、ttl、Frame.len可能要改名字
#network、ip.dst 、 ip.src 
#latitude、longtitude

#k-means: 群內距離短、群間距離大，取平均值找最佳的k
# cluster = KMeans(init='k-means++',n_init=10,max_iter=350,precompute_distances='auto',algorithm='auto',random_state=725035,n_clusters=2,n_jobs=-1,verbose=0) #11,305
# cluster.fit(train_X_PCA)

# ## 範例
# kmeans_fit = cluster.KMeans(n_clusters = 3).fit(iris_X)
# cluster_labels = kmeans_fit.labels_
# # 印出績效，選大值接近1者
# silhouette_avg = metrics.silhouette_score(iris_X, cluster_labels)
# print(silhouette_avg)
# plt.plot(range(2,20), silhouette_avg)